In [1]:
import pprint
import requests
import csv
import datetime
import pandas as pd
import time
import nltk
import re
from pandas.io.json import json_normalize
from sklearn.cluster import SpectralClustering
from math import sqrt, floor

pd.set_option("display.max_columns", 50)

In [28]:
results = requests.get("https://api.nytimes.com/svc/books/v3/lists.json?list=e-book-fiction&published-date=2017-02-01&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb")

In [29]:
results.status_code

200

In [50]:
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists.json"
LIST = "e-book-fiction"
API_KEY = 'keys.json'

bestsellers = []
weeks = []
num_calls_to_make = 1
weeks_per_call = 260  # Number of weeks to fetch per API call

for call_num in range(num_calls_to_make):
    ctr = call_num * weeks_per_call
    start_date = datetime.date(2017, 1, 25) - datetime.timedelta(weeks=ctr)
    end_date = start_date - datetime.timedelta(weeks=weeks_per_call)
    
    for i in range(weeks_per_call):
        curr_date = start_date - datetime.timedelta(weeks=i)
        print(curr_date)
        url = "%s?list=%s&published-date=%s&api-key=%s" % (BOOKS_ROOT, LIST, curr_date, API_KEY)
        url = url.strip()
        print(url)
        r = requests.get(url)
        
        if r.status_code == 200:
            results = r.json()
            for result in results['results']:
                for book_details in result['book_details']:
                    book_info = {
                        'title': book_details['title'],
                        'description': book_details['description'],
                        'contributor': book_details['contributor'],
                        'author': book_details['author'],
                        'contributor_note': book_details['contributor_note'],
                        'price': book_details['price'],
                        'age_group': book_details['age_group'],
                        'publisher': book_details['publisher'],
                        'primary_isbn13': book_details['primary_isbn13'],
                        'primary_isbn10': book_details['primary_isbn10'],
                        'rank': result['rank'],
                        'rank_last_week': result['rank_last_week'],
                        'weeks_on_list': result['weeks_on_list'],
                        'asterisk': result['asterisk'],
                        'dagger': result['dagger']
                    }
                    bestsellers.append(book_info)
                    weeks.append(str(curr_date))
                
        else:
            print("API call did not work!")
            print(r)
        
        time.sleep(15)


2017-01-25
https://api.nytimes.com/svc/books/v3/lists.json?list=e-book-fiction&published-date=2017-01-25&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2017-01-18
https://api.nytimes.com/svc/books/v3/lists.json?list=e-book-fiction&published-date=2017-01-18&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2017-01-11
https://api.nytimes.com/svc/books/v3/lists.json?list=e-book-fiction&published-date=2017-01-11&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2017-01-04
https://api.nytimes.com/svc/books/v3/lists.json?list=e-book-fiction&published-date=2017-01-04&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2016-12-28
https://api.nytimes.com/svc/books/v3/lists.json?list=e-book-fiction&published-date=2016-12-28&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2016-12-21
https://api.nytimes.com/svc/books/v3/lists.json?list=e-book-fiction&published-date=2016-12-21&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2016-12-14
https://api.nytimes.com/svc/books/v3/lists.json?list=e-book-fiction&published-date=2016-12-14&api-key=P1yDSg62Ayd

In [51]:
e_book_fiction = pd.json_normalize(bestsellers)

In [52]:
e_book_fiction.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
0,FULL PACKAGE,A man shares a cramped apartment with his frie...,by Lauren Blakely,Lauren Blakely,,0.00,,Lauren Blakely,A00B01MT5HMRV,None,1,0,1,0,0
1,THE GUERNSEY LITERARY AND POTATO PEEL PIE SOCIETY,"After World War II, a journalist travels to th...",by Mary Ann Shaffer and Annie Barrows,Mary Ann Shaffer and Annie Barrows,,0.00,,Dial,9780440337973,0440337976,2,0,2,0,0
2,THE WHISTLER,A whistleblower alerts a Florida investigator ...,by John Grisham,John Grisham,,0.00,,Doubleday,9780385541206,0385541201,3,4,12,0,0
3,RING OF FIRE,"Pike Logan, a member of a secret counterterror...",by Brad Taylor,Brad Taylor,,0.00,,Dutton,9781101984772,None,4,0,1,0,0
4,SMALL GREAT THINGS,"A medical crisis entangles a black nurse, a wh...",by Jodi Picoult,Jodi Picoult,,0.00,,Ballantine,9780345544964,034554496X,5,7,9,0,0


In [53]:
e_book_fiction.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
4820,UNFINISHED BUSINESS,Former high school sweethearts fall tentativel...,by Nora Roberts,Nora Roberts,,0.00,,Harlequin,9781459273702,None,16,0,19,0,0
4821,GOLDEN LIES,A bronze dragon found in an attic turns out to...,by Barbara Freethy,Barbara Freethy,,0.00,,Signet,A00B004Q9TI54,None,17,0,4,0,0
4822,WIFE BY WEDNESDAY,"A wealthy, titled Englishman who must marry to...",by Catherine Bybee,Catherine Bybee,,0.00,,Catherine Bybee,A00B005SJLB24,None,18,0,1,0,0
4823,THE 7TH MONTH,"D.D. Warren, a Boston detective in her seventh...",by Lisa Gardner,Lisa Gardner,,0.00,,Dutton,9781101572528,None,19,0,3,0,0
4824,ISLAND OF LOST GIRLS,A kidnapping stirs memories in a woman whose c...,by Jennifer McMahon,Jennifer McMahon,,0.00,,HarperCollins Publishers,9780061807589,0061807583,20,0,4,0,0


In [54]:
len(e_book_fiction)

4825

In [55]:
len(weeks)

4825

In [56]:
e_book_fiction['week'] = weeks

In [58]:
e_book_fiction.head()
e_book_fiction.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week
4820,UNFINISHED BUSINESS,Former high school sweethearts fall tentativel...,by Nora Roberts,Nora Roberts,,0.00,,Harlequin,9781459273702,None,16,0,19,0,0,2012-02-08
4821,GOLDEN LIES,A bronze dragon found in an attic turns out to...,by Barbara Freethy,Barbara Freethy,,0.00,,Signet,A00B004Q9TI54,None,17,0,4,0,0,2012-02-08
4822,WIFE BY WEDNESDAY,"A wealthy, titled Englishman who must marry to...",by Catherine Bybee,Catherine Bybee,,0.00,,Catherine Bybee,A00B005SJLB24,None,18,0,1,0,0,2012-02-08
4823,THE 7TH MONTH,"D.D. Warren, a Boston detective in her seventh...",by Lisa Gardner,Lisa Gardner,,0.00,,Dutton,9781101572528,None,19,0,3,0,0,2012-02-08
4824,ISLAND OF LOST GIRLS,A kidnapping stirs memories in a woman whose c...,by Jennifer McMahon,Jennifer McMahon,,0.00,,HarperCollins Publishers,9780061807589,0061807583,20,0,4,0,0,2012-02-08


In [59]:
e_book_fiction['rank'] = e_book_fiction['rank'].astype(str)
filter_15 = e_book_fiction['rank'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'])

In [60]:
e_book_fiction = e_book_fiction[filter_15]

In [62]:
e_book_fiction.head(20)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week
0,FULL PACKAGE,A man shares a cramped apartment with his frie...,by Lauren Blakely,Lauren Blakely,,0.00,,Lauren Blakely,A00B01MT5HMRV,None,1,0,1,0,0,2017-01-25
1,THE GUERNSEY LITERARY AND POTATO PEEL PIE SOCIETY,"After World War II, a journalist travels to th...",by Mary Ann Shaffer and Annie Barrows,Mary Ann Shaffer and Annie Barrows,,0.00,,Dial,9780440337973,0440337976,2,0,2,0,0,2017-01-25
2,THE WHISTLER,A whistleblower alerts a Florida investigator ...,by John Grisham,John Grisham,,0.00,,Doubleday,9780385541206,0385541201,3,4,12,0,0,2017-01-25
3,RING OF FIRE,"Pike Logan, a member of a secret counterterror...",by Brad Taylor,Brad Taylor,,0.00,,Dutton,9781101984772,None,4,0,1,0,0,2017-01-25
4,SMALL GREAT THINGS,"A medical crisis entangles a black nurse, a wh...",by Jodi Picoult,Jodi Picoult,,0.00,,Ballantine,9780345544964,034554496X,5,7,9,0,0,2017-01-25
5,SHELTER FOR ADELINE,A fireman must keep his overprotective nature ...,by Susan Stoker,Susan Stoker,,0.00,,Stoker Aces Production,A00B01MF62CN8,None,6,0,1,0,0,2017-01-25
6,A MAN CALLED OVE,"An angry old curmudgeon gets new neighbors, an...",by Fredrik Backman,Fredrik Backman,,0.00,,Atria,9781476738031,None,7,9,3,0,0,2017-01-25
7,THE MISTRESS,The beautiful mistress of a Russian oligarch f...,by Danielle Steel,Danielle Steel,,0.00,,Delacorte,9780425285350,0425285359,8,2,2,0,0,2017-01-25
8,THE GUESTS ON SOUTH BATTERY,Spirits invade the life of a Charleston realtor.,by Karen White,Karen White,,0.00,,Berkley,9780698193000,,9,0,1,0,0,2017-01-25
9,NO MAN'S LAND,"John Puller, a special agent with the Army, se...",by David Baldacci,David Baldacci,,0.00,,Grand Central,9781455586493,1455586498,10,0,8,0,0,2017-01-25


In [63]:
len(e_book_fiction)

3901

In [64]:
#and a column indicating which list these come from
k2 = 3901
weekly_fiction_list = ['E-Book Fiction']
nyt_list_weekly_fiction = []

for element in weekly_fiction_list:
    for i in range(k2):
        nyt_list_weekly_fiction.append(element)

len(nyt_list_weekly_fiction)

3901

In [65]:
#add nyt_list as a column
e_book_fiction['nyt_list'] = nyt_list_weekly_fiction

#and check to make sure it worked correctly
e_book_fiction.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week,nyt_list
4815,THE MILL RIVER RECLUSE,"Only one man knows an abused widow's secret, w...",by Darcie Chan,Darcie Chan,,0.00,,Darcie Chan,2940012964205,None,11,0,24,0,0,2012-02-08,E-Book Fiction
4816,11/22/63,An English teacher travels back to 1958 by way...,by Stephen King,Stephen King,,0.00,,Scribner,9781451627305,None,12,0,12,0,0,2012-02-08,E-Book Fiction
4817,A LADY NEVER SURRENDERS,When the youngest Sharpe sister hatches a plan...,by Sabrina Jeffries,Sabrina Jeffries,,0.00,,Pocket Books,9781451642490,1451642490,13,0,1,0,0,2012-02-08,E-Book Fiction
4818,WHAT IT WAS,"In the summer of 1972, Derek Strange has left ...",by George Pelecanos,George Pelecanos,,0.00,,"Little, Brown & Company",9780316209557,0316209554,14,0,1,0,0,2012-02-08,E-Book Fiction
4819,WATER FOR ELEPHANTS,A young veterinary student — and an elephant —...,by Sara Gruen,Sara Gruen,,0.00,,Algonquin,9781565125858,1565125851,15,0,46,0,0,2012-02-08,E-Book Fiction


In [66]:
#save to csv
e_book_fiction.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\data\\2012to2017_e_book_fiction.csv", index=False)